In [11]:
import string
import re
from numpy import array, argmax, random, take
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras_preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers

import matplotlib.pyplot as plt
#from keras.layers.normalization.layer_normalization import *
#from keras.layers.normalization.batch_normalization import *
%matplotlib inline
pd.set_option('display.max_colwidth', 200)

2023-10-16 09:15:39.876209: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-16 09:15:39.877731: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 09:15:39.910063: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 09:15:39.910952: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 09:15:40.592014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [7]:
def read_text(filename):
        # open the file
        file = open(filename, mode='rt', encoding='utf-8')
        
        # read all text
        text = file.read()
        file.close()
        return text

In [8]:
def to_lines(text):
    sents = text.strip().split('\n')
    sents = [i.split('\t') for i in sents]
    return sents

In [9]:
import pandas as pd
data = read_text("For complex questions.txt")
print(data)

I received the 'VAX_NAME' vaccine on 'VAX_DATE' and noticed 'SYMPTOM_TEXT'.
After my 'VAX_TYPE' injection on 'VAX_DATE', my symptoms worsened, prompting an 'ER_VISIT'.
I'm curious if my gender ('SEX') could influence the severity of 'SYMPTOM_TEXT' following the 'VAX_NAME' vaccine.
I'm a 'AGE_YRS' patient who experienced 'SYMPTOM_TEXT' after the 'VAX_NAME' vaccine on 'VAX_DATE'.
Following my 'VAX_NAME' vaccine on 'VAX_DATE', my condition deteriorated, leading to a hospital admission.
I'm wondering if there's a connection between my 'SYMPTOM_TEXT' and 'DISABLE' after receiving the 'VAX_TYPE' vaccine.
I had an 'OFC_VISIT' due to 'SYMPTOM_TEXT' after the 'VAX_TYPE' injection on 'VAX_DATE'.
My allergic reaction to 'ALLERGIES' after the 'VAX_NAME' vaccine on 'VAX_DATE' resulted in an 'ER_VISIT'.
After my 'VAX_TYPE' injection on 'VAX_DATE', I experienced 'SYMPTOM_TEXT' and needed an 'OFC_VISIT'.
I'm interested in understanding if 'VAX_NAME' can affect the severity of 'SYMPTOM_TEXT' in individ

In [13]:
C_q = to_lines(data)
C_q = array(C_q)
C_q 

array([["I received the 'VAX_NAME' vaccine on 'VAX_DATE' and noticed 'SYMPTOM_TEXT'."],
       ["After my 'VAX_TYPE' injection on 'VAX_DATE', my symptoms worsened, prompting an 'ER_VISIT'."],
       ["I'm curious if my gender ('SEX') could influence the severity of 'SYMPTOM_TEXT' following the 'VAX_NAME' vaccine."],
       ["I'm a 'AGE_YRS' patient who experienced 'SYMPTOM_TEXT' after the 'VAX_NAME' vaccine on 'VAX_DATE'."],
       ["Following my 'VAX_NAME' vaccine on 'VAX_DATE', my condition deteriorated, leading to a hospital admission."],
       ["I'm wondering if there's a connection between my 'SYMPTOM_TEXT' and 'DISABLE' after receiving the 'VAX_TYPE' vaccine."],
       ["I had an 'OFC_VISIT' due to 'SYMPTOM_TEXT' after the 'VAX_TYPE' injection on 'VAX_DATE'."],
       ["My allergic reaction to 'ALLERGIES' after the 'VAX_NAME' vaccine on 'VAX_DATE' resulted in an 'ER_VISIT'."],
       ["After my 'VAX_TYPE' injection on 'VAX_DATE', I experienced 'SYMPTOM_TEXT' and needed an 'OFC_V